<a href="https://colab.research.google.com/github/mohamedhassan279/Markov-Decision-Process/blob/main/Markov_Decision_Process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### At the begining, We have represented Our state to be **(`the position of the agent (row,column) in the 3x3 MDP environment`)**.

# Project Set Up Modules , Classes and Functions.

### First, we have imported some modules to be used in the assignment and these modules are:
1- `Enum` module is imported for creating enumerations. We use this module to enumerate the possible directions for the agent.<br>
2- `copy` module is imported for creating deep copies of objects. We use this module to copy the *"old_policy"* array into new array called *"new_policy"*.<br>
3- `numpy` library is imported as `np` for numerical operations.<br>
### Some global variables which are used in the assignment:
1- The MDP environment is represented by a 3x3 grid (`world` array) where each element denotes the reward for being in that state. `-1` represents an intermediate state, and (`0` and `10`) represents End states.<br>
Note: "the state's reward can hold fractions so that the array datatype is `float64` "<br>
2- Parameters such as the discount factor (`gamma`), convergence threshold (`epsilon`), success probability (`success_prop`), and failure probability (`fail_prop`) are initialized.

In [ ]:
from enum import Enum
import copy
import numpy as np

world = np.array([[0, -1, 10], [-1, -1, -1], [-1, -1, -1]], dtype=np.float64)
gamma = 0.99
epsilon = 1e-9
success_prop = 0.8
fail_prop = 0.1

### This code defines an enumeration Direction with members representing different movement directions in a grid or 2D space.
**Possible Directions:**<br>
Suppose my position is (row,column)
1. (0,0) → i.e. (row + 0, column + 0) means it is the same as the original position.
2. (-1,0) → i.e. (row - 1, column + 0) means it is the **up** position.
3. (1,0) → i.e. (row + 1, column + 0) means it is the **down** position.
4. (0,-1) → i.e. (row + 0, column - 1) means it is the **left** position.
5. (0,1) → i.e. (row + 0, column + 1) means it is the **right** position.

In [ ]:
class Direction(Enum):
    STAY = (0, 0)
    UP = (-1, 0)
    DOWN = (1, 0)
    LEFT = (0, -1)
    RIGHT = (0, 1)

### Utility Functions.<br>
1) `isEnd` function checks if a given position corresponds to one of the specified end states, the end state here is (the position (0,0) which is "r" cell) and (position (0,2) which is the "+10" cell).
<br>2) `isWall` function checks if a given position is outside the boundaries of a 3x3 grid (considered a wall),and this can be obtained by the following cases:
1. if the `i` has negative value or not indicating that the **up** position is a wall.
2. if the `i` has value>2 or not indicating that the **down** position is a wall.
3. if the `j` has negative value or not indicating that the **left** position is a wall.
4. if the `j` has value>2 or not indicating that the **right** position is a wall.

In [ ]:
def isEnd(i, j):
    return (i == 0 and j == 2) or (i == 0 and j == 0)


def isWall(i, j):
    return i < 0 or i > 2 or j < 0 or j > 2

### Utility Functions.
1- (`get_successor`) function, takes the current position (i, j) and a direction of movement as arguments.<br>It generates successor states based on the current position and movement direction, considering potential moves in the **specified direction**, the **perpendicular direction**, and the **opposite direction**. The function returns a (`list of successor states`), where each successor state is represented as a tuple (new_i, new_j, probability), indicating the **new position** and the **probability** of transitioning to that state.<br>
2- (`reward`) function, takes the row index i and column index j as arguments, and retrieves (`the reward associated with the specified position`) in the Markov Decision Process (MDP) world.

In [ ]:
def get_successor(i, j, direction):
    successors = []
    if not isWall(i + direction.value[0], j + direction.value[1]):
        successors.append((i + direction.value[0], j + direction.value[1], success_prop))
    else:
        successors.append((i, j, success_prop))
    if not isWall(i + direction.value[1], j + direction.value[0]):
        successors.append((i + direction.value[1], j + direction.value[0], fail_prop))
    else:
        successors.append((i, j, fail_prop))
    if not isWall(i - direction.value[1], j - direction.value[0]):
        successors.append((i - direction.value[1], j - direction.value[0], fail_prop))
    else:
        successors.append((i, j, fail_prop))
    return successors


def reward(i, j):
    return world[i][j]

# Main Functions

### (`value_iteration`) function
Performs the value iteration algorithm to find the optimal value function and policy for a given Markov Decision Process (MDP).

    Returns:
    1. V (numpy.ndarray): 2D array representing the optimal value function for each state in the MDP.
    2. optimal_policy (numpy.ndarray): 2D array representing the optimal policy for each state in the MDP.
---
It works as following:
1. Initialize the V array with zeros.
2. Initialize the optimal_policy array with STAY actions.
3. Enter an infinite loop until the maximum change in values (delta) is small enough (convergence to epsilon).
4. Set delta as the maximum absolute change in the values of states between consecutive iterations.
5. State Value Update:
  * Nested loops iterate over all states in the MDP grid.
  * Skip end states using the isEnd function.
  * v: Store the current value of the state.
  * Find the maximum value among possible actions using the Bellman equation.
  * Update the state value (V[i][j]) and optimal policy based on the action with the maximum value.
  * Update the change in value (delta).
6. Convergence Check:
  * If the maximum change in value (delta) is below the convergence threshold (epsilon), break out of the infinite loop.
7. Returns:
  * V (numpy.ndarray): 2D array representing the optimal value function for each state in the MDP.
  * optimal_policy (numpy.ndarray): 2D array representing the optimal policy for each state in the MDP.

In [ ]:
def value_iteration():
    V = np.zeros_like(world, dtype=np.float64)
    optimal_policy = np.array([[Direction.STAY for _ in range(3)] for _ in range(3)])
    iterations = 0
    while True:
        iterations += 1
        delta = 0  # delta = max(|v - v'|)
        for i in range(3):
            for j in range(3):
                if isEnd(i, j):
                    V[i][j] = reward(i, j)
                    continue
                v = V[i][j]
                max_value = float('-inf')
                for direction in Direction:
                    successors = get_successor(i, j, direction)
                    value = 0
                    for x, y, prop in successors:
                        value += prop * (reward(i, j) + gamma * V[x][y])

                    if value > max_value:
                        max_value = value
                        optimal_policy[i][j] = direction
                V[i][j] = max_value
                delta = max(delta, float(abs(v - V[i][j])))
        if delta < epsilon:
            break
    print("Number of iterations: " + str(iterations))
    return V, optimal_policy

### (`print_policy`) function.<br>
Prints a visual representation of the policy.

    Parameters:
    - policy (numpy.ndarray): 2D array representing the policy with Direction enums.

    Prints:
    - Visual representation of the policy using arrows (↑, ↓, ←, →) or 'E' for 'STAY'.
---
It works as following:
* Printing the Policy:
  1. Nested loops iterate over rows and columns of the `policy` array.
  2. Check the value of each element in the array and print the corresponding arrow ('↑', '↓', '←', '→') or 'E' for 'STAY' direction.
  3. The printed arrows represent the recommended actions in the policy for each state.

In [ ]:
def print_policy(policy):
    for i in range(3):
        for j in range(3):
            if policy[i][j] == Direction.UP:
                print('↑', end=' ')
            elif policy[i][j] == Direction.DOWN:
                print('↓', end=' ')
            elif policy[i][j] == Direction.LEFT:
                print('←', end=' ')
            elif policy[i][j] == Direction.RIGHT:
                print('→', end=' ')
            else:
                print('E', end=' ')
        print()

### 1. Implement value iteration for this world for each value of r below:
  * r = 100
  * r = 3
  * r = 0
  * r = -3

### 2. Show the policy obtained in each case.




In [ ]:
for r in [100, 3, 0, -3]:
    world[0][0] = r
    optimal_values, optimal_policy = value_iteration()
    print('r = ' + str(r))
    print("optimal values:-")
    print(optimal_values)
    print()
    print("optimal policy:-")
    print_policy(optimal_policy)
    print('----------------------------------------------------------------------------------')

Number of iterations: 25
r = 100
optimal values:-
[[100.          97.20352595  10.        ]
 [ 97.20352595  94.75128165  88.204261  ]
 [ 94.48183416  92.35292957  89.76219984]]

optimal policy:-
E ← E 
↑ ← ↓ 
↑ ← ← 
----------------------------------------------------------------------------------
Number of iterations: 23
r = 3
optimal values:-
[[ 3.          8.46193927 10.        ]
 [ 5.38356505  7.11320491  8.46193927]
 [ 4.57481583  5.79411126  6.96500879]]

optimal policy:-
E → E 
→ → ↑ 
→ → ↑ 
----------------------------------------------------------------------------------
Number of iterations: 23
r = 0
optimal values:-
[[ 0.          8.46193927 10.        ]
 [ 5.08329878  7.11320491  8.46193927]
 [ 4.5418232   5.79411126  6.96500879]]

optimal policy:-
E → E 
→ → ↑ 
→ → ↑ 
----------------------------------------------------------------------------------
Number of iterations: 23
r = -3
optimal values:-
[[-3.          8.46193927 10.        ]
 [ 4.78303251  7.11320491  8.46193927

### Explain intuitively why the value of r leads to each policy.
    For r = 100:
        Intuition:
            A very high positive reward (r = 100) creates a strong incentive for the agent to reach the goal state "r" -rather than the another goal in position (0,2)- as quickly as possible.
            The agent prefers actions that lead to the goal "r", even if it incurs some negative rewards along the way.
            the cell under positive 10 suggests moving down instead of taking the shortcut to reach r by moving left as moving left may lead to move up with probability 0.1 and end the game with reaching the r = 100.
        Policy Explanation:
            The optimal policy involves moving to r as fast and as safe as possible by the directions resulted above.

    For r = 3:
        Intuition:
            A moderate positive reward (r = 3) but less than the goal at position (0,2) which reward is "+10" this will make the agent still to be encouraged reaching the goal at position (0,2) more than reaching the goal "r" but with less urgency compared to the high positive reward case which is "100" in the previous case.
            The agent aims to reach the goal while considering a more balanced trade-off between positive and negative rewards.
        Policy Explanation:
            The optimal policy involves a combination of moving right ('→') and upwards ('↑') to reach the goal while avoiding negative rewards to reach position (0,2).

    For r = 0:
        Intuition:
            A zero reward (r = 0) implies a neutral state with no inherent motivation to reach the goal at "r" quickly but reaching the goal at position (0,2) is much more preferable.
            The agent aims to reach the goal at position (0,2) by finding a path to this goal while minimizing negative rewards.
        Policy Explanation:
            The optimal policy is to get to the goal with reward 10 because it's more optimal than having a zero reward at (0,0), so we find that each cell aims to the cell at (0,2)

    For r = -3:
        Intuition:
            A negative reward (r = -3) discourages reaching the goal at this position but much much more encouraged to reach the goal at position (0,2) with reward "+10", and the agent seeks paths with fewer negative rewards to reach position (0,2).
        Policy Explanation:
            The optimal policy involves moving right ('→') and upwards ('↑') to explore the environment while avoiding the goal with "-3" reward but finding path to reach position (0,2) with value "+10" minimizing the cumulative negative reward.

Overall Intuition:

    The value of r serves as a motivational factor for the agent, influencing its trade-off between reaching which goal (goal with reward r or with reward +10) and avoiding negative rewards.
    Higher positive rewards lead to more aggressive policies, prioritizing goal attainment.
    Zero or negative rewards lead to more cautious policies, emphasizing the avoidance of negative rewards and exploring alternative paths to the goal.

# Bounus:
Find the optimal policy for each of the previous cases of `r` using Policy Iteration algorithm. You may start the algorithm with a randomly generated policy.

### `generate_randomized_policy` Function.
    Return the final randomized_policy matrix, representing a randomized policy for non-fixed positions in the MDP environment.
---
It works as following:
1. Policy Initialization:
    * Initialize a 3x3 matrix called `randomized_policy` to represent a randomized policy. All directions are initially set to (`Direction.STAY`).

2. Direction Definition:
    * Create a list directions containing all possible directions except (`Direction.STAY`).

3. Randomization Loop:
    * Iterate through each position in the `randomized_policy` matrix.
    * For each position, if it is not one of the fixed positions (0, 0) or (0, 2):
      * Exclude the (`Direction.STAY`) option.
      * Randomly choose a direction from the list of possible directions (excluding STAY).
      * Assign the randomly chosen direction to the current position in the (`randomized_policy`) matrix.

4. Return Randomized Policy:
    * Return the final (`randomized_policy`) matrix, representing a randomized policy for non-fixed positions in the MDP environment.

In [ ]:
def generate_randomized_policy():
    randomized_policy = np.array([[Direction.STAY for _ in range(3)] for _ in range(3)])
    directions = [d for d in Direction if d != Direction.STAY]
    # Iterate through the array and randomize directions
    for i in range(randomized_policy.shape[0]):
        for j in range(randomized_policy.shape[1]):
            if (i, j) not in [(0, 0), (0, 2)]:
                # Exclude STAY and randomize direction
                randomized_policy[i, j] = np.random.choice(directions)
    return randomized_policy

### (`policy_evaluation`) Function
Evaluates the given policy by iteratively updating state values.

    Parameters:
    - policy (numpy.ndarray): 2D array representing the policy with Direction enums.

    Returns:
    - numpy.ndarray: 2D array containing the state values after policy evaluation.
---
It works as following:
  1. Initialization:
      * V: Initialize a 3x3 matrix V representing state values to zeros.

  2. Policy Evaluation Loop:
        Enter a loop for policy evaluation until convergence.
        delta: Initialize the change in value to zero.

  3. State Value Update Loop:
        Nested loops iterate over all states in the MDP grid.
        Skip end states using isEnd function.
        v: Store the current value of the state.
        action: Get the current action according to the input policy.

  4. Calculate Successor Values:
        Get successor states based on the current action using the get_successor function.
        Calculate the value for the current state using the Bellman equation.

  5. Update State Values:
        Update the value of the current state (V[i][j]) based on the calculated value.

  6. Convergence Check:
        Update the change in value (delta) based on the maximum change observed during the iteration.
        Check for convergence, i.e., whether the maximum change in value is below the convergence threshold (epsilon).
        If convergence is achieved, exit the loop.
  7. Returns:
    * 2D array containing the state values after policy evaluation.

In [ ]:
def policy_evaluation(policy):
    V = np.zeros_like(world, dtype=np.float64)
    while True:
        delta = 0  # delta = max(|v - v'|)
        for i in range(3):
            for j in range(3):
                if isEnd(i, j):
                    V[i][j] = reward(i, j)
                    continue
                v = V[i][j]
                successors = get_successor(i, j, policy[i][j])
                value = 0
                for x, y, prop in successors:
                    value += prop * (reward(i, j) + gamma * V[x][y])
                V[i][j] = value
                delta = max(delta, float(abs(v - V[i][j])))
        if delta < epsilon:
            break
    return V

### (`policy_improvement`) Function:
Improves the given policy based on the current state values.

    Parameters:
    - V (numpy.ndarray): 2D array representing state values.
    - policy (numpy.ndarray): 2D array representing the current policy with Direction enums.

    Modifies:
    - policy (numpy.ndarray): The input policy is modified based on the state values.
---
It works as following:
1. Iterate Over States:
    * Nested loops iterate over all states in the MDP grid.

2. Skip End States:
    * If the current state is an end state, skip it using the (`isEnd`) function.

3. Policy Improvement Loop:
    * For each non-end state, initialize max_value to negative infinity.

4. Iterate Over Actions (`Directions`):
    * Nested loop iterates over all possible actions (`directions`) using the Direction enumeration.

5. Calculate Expected Value:
    * For each action, get the successor states based on the current action using (`get_successor`).
    * Calculate the expected value for the current action using the Bellman equation.

6. Update Policy:
    * If the calculated value is greater than the current maximum value:
      * Update the maximum value.
      * Update the policy for the current state to the current action (`direction`).

7. Modified Policy:
    * After iterating over all actions, the policy for the current state is updated to the action that provides the maximum expected value.

In [ ]:
def policy_improvement(V, policy):
    for i in range(3):
        for j in range(3):
            if isEnd(i, j):
                continue
            max_value = float('-inf')
            for direction in Direction:
                successors = get_successor(i, j, direction)
                value = 0
                for x, y, prop in successors:
                    value += prop * (reward(i, j) + gamma * V[x][y])

                if value > max_value:
                    max_value = value
                    policy[i][j] = direction

### (`policy_iteration`) Function:
Executes the policy iteration algorithm to find an optimal policy.

    Returns:
    - V (numpy.ndarray): 2D array representing the final state values.
    - policy (numpy.ndarray): 2D array representing the optimal policy.
---
It works as following:
1. Policy Initialization:
    * Initialize (`old_policy`) with a randomized policy using the generate_randomized_policy function.
    * Initialize an iteration counter i to 1.

2. Iteration Loop:
    * Enter a loop for policy iteration until convergence.

3. Policy Evaluation:
    * Evaluate the state values (V) based on the current policy using the (`policy_evaluation`) function.

4. Policy Improvement:
    * Create a deep copy of the current policy to (`new_policy`).
    * Print the current iteration number and the policy for the iteration using (`print_policy`).
    * Improve the policy based on the current state values using the (`policy_improvement`) function.

5. Convergence Check:
    * Check for convergence by comparing the old and new policies using np.array_equal.
    * If the policies are equal, break out of the loop.

6. Update Old Policy:
    * If not converged, update the (`old_policy`) with the new_policy for the next iteration.

7. Return Result:
    * Return the final state values (`V`) and the optimal policy (`old_policy`).

In [ ]:
def policy_iteration():
    old_policy = generate_randomized_policy()
    i = 1
    while True:
        V = policy_evaluation(old_policy)
        new_policy = copy.deepcopy(old_policy)
        print("iteration ", i)
        i += 1
        print_policy(new_policy)
        policy_improvement(V, new_policy)
        if np.array_equal(old_policy, new_policy):
            break
        old_policy = copy.deepcopy(new_policy)
    return V, old_policy

### Find the optimal policy for each of the cases of r {100,3,0,-3} using Policy Iteration algorithm.
    This code snippet appears to perform policy iteration for different reward values in a Markov Decision Process (MDP) environment and prints the resulting optimal values and policies.

In [ ]:
for r in [100, 3, 0, -3]:
    world[0][0] = r
    optimal_values, optimal_policy = policy_iteration()
    print('\nr = ' + str(r))
    print("optimal values:-")
    print(optimal_values)
    print()
    print("optimal policy:-")
    print_policy(optimal_policy)
    print('----------------------------------------------------------------------------------')

iteration  1
E ↑ E 
↓ → ← 
↓ ↓ ↓ 
iteration  2
E ← E 
↑ ↑ ↑ 
↑ ↑ ↑ 
iteration  3
E ← E 
↑ ← ← 
↑ ← ← 
iteration  4
E ← E 
↑ ← ↓ 
↑ ← ← 

r = 100
optimal values:-
[[100.          97.20352595  10.        ]
 [ 97.20352595  94.75128165  88.204261  ]
 [ 94.48183416  92.35292957  89.76219984]]

optimal policy:-
E ← E 
↑ ← ↓ 
↑ ← ← 
----------------------------------------------------------------------------------
iteration  1
E ↓ E 
← ↑ ↓ 
→ ↓ → 
iteration  2
E → E 
↑ ↑ ↑ 
↑ ↑ ↑ 
iteration  3
E → E 
→ → ↑ 
→ → ↑ 

r = 3
optimal values:-
[[ 3.          8.46193927 10.        ]
 [ 5.38356505  7.11320491  8.46193927]
 [ 4.57481583  5.79411126  6.96500879]]

optimal policy:-
E → E 
→ → ↑ 
→ → ↑ 
----------------------------------------------------------------------------------
iteration  1
E ↑ E 
↑ → ↑ 
↑ ↑ ↓ 
iteration  2
E → E 
→ → ↑ 
→ ↑ ↑ 
iteration  3
E → E 
→ → ↑ 
→ → ↑ 

r = 0
optimal values:-
[[ 0.          8.46193927 10.        ]
 [ 5.08329878  7.11320491  8.46193927]
 [ 4.5418232   5.79